In [1]:
import os, gc, pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style
from matplotlib.ticker import MaxNLocator

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import scipy
import scipy.sparse

import gc
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    if y_true.shape != y_pred.shape: raise ValueError("Shapes are different.")
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)


# Preprocessing and cross-validation

We first load all of the training input data for Multiome. It should take less than a minute.

In [3]:
%%time
train_inputs = scipy.sparse.load_npz("../input/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_values.sparse.npz")

CPU times: user 35.8 s, sys: 3.35 s, total: 39.1 s
Wall time: 1min 1s


In [4]:
train_inputs = train_inputs.astype('float16', copy=False)

## PCA / TruncatedSVD
It is not possible to directly apply PCA to a sparse matrix, because PCA has to first "center" the data, which destroys the sparsity. This is why we apply `TruncatedSVD` instead (which is pretty much "PCA without centering"). It might be better to normalize the data a bit more here, but we will keep it simple.

In [5]:
%%time
pca = TruncatedSVD(n_components=50, random_state=42)
train_inputs = pca.fit_transform(train_inputs)
print(pca.explained_variance_ratio_.sum())

0.0087636635
CPU times: user 9min 27s, sys: 14.3 s, total: 9min 42s
Wall time: 9min 33s


In [6]:
%%time
train_targets = scipy.sparse.load_npz("../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_values.sparse.npz")

CPU times: user 16 s, sys: 1.13 s, total: 17.1 s
Wall time: 25 s


In [7]:
%%time
pca2 = TruncatedSVD(n_components=50, random_state=42)
train_target = pca2.fit_transform(train_targets)
print(pca2.explained_variance_ratio_.sum())

0.10559903
CPU times: user 2min 18s, sys: 3.78 s, total: 2min 21s
Wall time: 2min 16s


In [8]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.kernel_ridge import KernelRidge
kernel = RBF(length_scale = 10)
krr = KernelRidge(alpha=0.2, kernel=kernel) # alpha range: 0.1-1.0

In [9]:
n = 5

In [10]:
np.random.seed(42)
all_row_indices = np.arange(train_inputs.shape[0])
np.random.shuffle(all_row_indices)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

index = 0
score = []

d = train_inputs.shape[0]//n
for i in range(0, n*d, d):
    print(f'start [{i}:{i+d}]')
    ind = all_row_indices[i:i+d]    
    for idx_tr, idx_va in kf.split(ind):
        X = train_inputs[ind]
        Y = train_target[ind] #.todense()
        Yva = train_targets[ind][idx_va]
        Xtr, Xva = X[idx_tr], X[idx_va]
        Ytr = Y[idx_tr]
        del X, Y
        gc.collect()
        print('Train...')
        model = krr #Ridge(copy_X=False)
        model.fit(Xtr, Ytr)
        del Xtr, Ytr
        gc.collect()
        s = correlation_score(Yva.todense(), model.predict(Xva)@pca2.components_)
        score.append(s)
        print(index, s)
        del Xva, Yva
        gc.collect()
        pkl_filename = f"model{index:02d}.pkl"
        index += 1
        with open(pkl_filename, 'wb') as file:
            pickle.dump(model, file)
    gc.collect()

start [0:21188]
Train...
0 0.6677549854475622
Train...
1 0.6678806945487705
Train...
2 0.66722359949446
Train...
3 0.6691273985274065
Train...
4 0.6688757779556356
start [21188:42376]
Train...
5 0.6681304955878865
Train...
6 0.6669767685373553
Train...
7 0.6689315401457525
Train...
8 0.6673662296332239
Train...
9 0.6673014971929867
start [42376:63564]
Train...
10 0.6692023929247037
Train...
11 0.6680836834719595
Train...
12 0.6670938440319344
Train...
13 0.6690746005909588
Train...
14 0.6672494652037124
start [63564:84752]
Train...
15 0.6680941619026332
Train...
16 0.666380281043041
Train...
17 0.6680227037423783
Train...
18 0.669842331257094
Train...
19 0.668893808037132
start [84752:105940]
Train...
20 0.6684693677006818
Train...
21 0.6696162231916681
Train...
22 0.6676087065943052
Train...
23 0.6674222413444935
Train...
24 0.6679624946019537


In [11]:
del train_target, train_inputs, train_targets
gc.collect()

21

# Predicting

In [12]:
%%time
multi_test_x = scipy.sparse.load_npz("../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_values.sparse.npz")
multi_test_x = pca.transform(multi_test_x)

CPU times: user 43.7 s, sys: 1.62 s, total: 45.3 s
Wall time: 1min 1s


In [14]:
test_len = multi_test_x.shape[0]
d = test_len//n
x = []
for i in range(n):
    x.append(multi_test_x[i*d:i*d+d])
del multi_test_x
gc.collect()

103

In [15]:
index

25

In [16]:
preds = np.zeros((test_len, 23418), dtype='float16')
for i,xx in enumerate(x):
    for ind in range(index):
        print(ind, end=' ')
        with open(f'model{ind:02}.pkl', 'rb') as file:
            model = pickle.load(file)
        preds[i*d:i*d+d,:] += (model.predict(xx)@pca2.components_)/index
        gc.collect()
    print('')
    del xx
gc.collect()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 


0

In [17]:
del x
gc.collect()

21

In [18]:
np.save('preds.npy', preds)

In [19]:
preds = preds.astype('float16', copy=False)

# Creating submission

We load the cells that will have to appear in submission.

In [20]:
%%time
# Read the table of rows and columns required for submission
eval_ids = pd.read_parquet("../input/multimodal-single-cell-as-sparse-matrix/evaluation.parquet")
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

CPU times: user 28.3 s, sys: 11 s, total: 39.2 s
Wall time: 33.7 s


In [21]:
# Prepare an empty series which will be filled with predictions
submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

We load the `index`  and `columns` of the original dataframe, as we need them to make the submission.

In [22]:
%%time
y_columns = np.load("../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_idxcol.npz",
                   allow_pickle=True)["columns"]

test_index = np.load("../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz",
                    allow_pickle=True)["index"]

CPU times: user 32.7 ms, sys: 6.64 ms, total: 39.3 ms
Wall time: 101 ms


We assign the predicted values to the correct row in the submission file.

In [23]:
cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
assert len(cell_dict)  == len(test_index)

gene_dict = dict((k,v) for v,k in enumerate(y_columns))
assert len(gene_dict) == len(y_columns)

In [24]:
eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

In [25]:
submission.iloc[valid_multi_rows] = preds[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

In [26]:
del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
gc.collect()

21

In [27]:
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86                    NaN
1         c2150f55becb  CD274                   NaN
2         c2150f55becb  CD270                   NaN
3         c2150f55becb  CD155                   NaN
4         c2150f55becb  CD112                   NaN
                                             ...   
65744175  2c53aa67933d  ENSG00000134419    6.664062
65744176  2c53aa67933d  ENSG00000186862    0.043762
65744177  2c53aa67933d  ENSG00000170959    0.053680
65744178  2c53aa67933d  ENSG00000107874    1.255859
65744179  2c53aa67933d  ENSG00000166012    5.785156
Name: target, Length: 65744180, dtype: float32

# Merging with CITEseq predictions

In [28]:
submission.reset_index(drop=True, inplace=True)
submission.index.name = 'row_id'

In [29]:
cite_submission = pd.read_csv("../input/msci-citeseq-keras-quickstart/submission.csv")
cite_submission = cite_submission.set_index("row_id")
cite_submission = cite_submission["target"]

In [30]:
submission[submission.isnull()] = cite_submission[submission.isnull()]

In [31]:
submission

row_id
0           0.094605
1          -0.162362
2          -0.405332
3          -0.302582
4           1.114355
              ...   
65744175    6.664062
65744176    0.043762
65744177    0.053680
65744178    1.255859
65744179    5.785156
Name: target, Length: 65744180, dtype: float32

In [32]:
submission.isnull().any()

False

In [33]:
submission.to_csv("submission.csv")